In [ ]:
import pandas as pd
import ast
# Specify the file path and encoding
file_path = 'hebrew_verbs.csv'
opath = 'hebrew_verbs_agg.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# filter by roots of len 3, pa'al, and part of speech
df['Root'] = df['Root'].apply(ast.literal_eval)
df = df[df['Root'].apply(lambda x: len(x) == 3)]
df = df[df['Binyan'].apply(lambda x: x == "pa'al")]
df = df[df['Part of speech'].apply(lambda x: x == "Verb")]

In [ ]:
def replace_final_form(hebrew_chars):
    # Define mapping of normal form characters to their final forms
    # reversed due to hebrew direction
    final_form_mapping = {
        "ץ": "צ",
        "ף": "פ",
        "ן": "נ",
        "ם": "מ",
        "ך": "כ"
    }
    
    # Replace final forms in the list
    return ''.join([final_form_mapping.get(char, char) for char in hebrew_chars])



In [ ]:
import networkx as nx

G = nx.Graph()

for index, row in df.iterrows():
    root = ''.join(row['Root'])
    word = row['Word (he)']
    meaning = row['Meaning']
        
    G.add_node(root, title=meaning, word=word)
    

In [ ]:
import itertools
from Levenshtein import distance

for n , m in itertools.combinations(G.nodes(), 2):
    # dist = distance(replace_final_form(n) , replace_final_form(m))
    # if dist <= 1:
    #     G.add_edge(n, m)
    if set(replace_final_form(n)) == set(replace_final_form(m)):
        G.add_edge(n, m)


In [ ]:
nx.write_gexf(G, "roots_verb_paal_3char.gexf")

In [ ]:
from pyvis.network import Network

# Create a new pyvis network
net = Network(notebook=True, bgcolor="#222222", font_color="white", height="100vh")

# Load the graph from the GEXF file
# G = nx.read_gexf("roots_verb_paal_3char.gexf")

# filter degree

nonzero_degree_nodes = [node for node, degree in G.degree if degree != 0]

# Create a new graph with only nodes of nonzero degree
G_filtered = G.subgraph(nonzero_degree_nodes)

net.from_nx(G_filtered)



# Set some options for the network
net.options = {
    "nodes": {
        "font": {
            "size": 36,
            "face": "tahoma"
        },
        "shape": "dot",
        "size": 10,
        "color": {
            "border": "#2B7CE9",  
            "background": "#666", 
            "highlight": {
                "border": "#2B7CE9",  
                "background": "#848484" 
            }
        },
        "shadow": {
            "enabled": True,
            "size": 15
        },
        "title": "meaning"  # Hover attribute to display semantic meaning
    },
    "edges": {
        "color": {
            "color": "#ccc", 
            "highlight": "#848484" 
        }
    },
    "interaction": {
        "hover": True,
        "navigationButtons": True,
        "zoomView": True,
        "dragView": True
    },
    "physics": {
        "enabled": True
    },
}

# Show the network
net.show("network.html")